In [9]:
# %%
import sys, os
try:
    # ✅ Running from a Python script (.py file)
    TOOLS_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), "..",))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    TOOLS_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

SRC_PATH = os.path.join(TOOLS_PATH)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src


In [2]:
# 2) Import LangChain and your tools
from langchain_openai import OpenAI
from langchain.agents import create_react_agent
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import ToolNode
from langgraph.types import Command
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [3]:
# 🔁 Import all tools from registry
from tools.qdrant_tools_registry import qdrant_search_tool

🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src
📌 Collection Name: indian_startups


In [4]:
# Define tools for the enhancer agent
qdrant_agent_tools = [
    qdrant_search_tool,
]

# Define tool names for the agent
tool_names = [tool.name for tool in qdrant_agent_tools]

# Define the tool descriptions
tool_descriptions = [tool.description for tool in qdrant_agent_tools]

# Build readable tool help text for the prompt
tool_help_text = "\n".join(
    [f"{i+1}. {tool.name} - {tool.description}" for i, tool in enumerate(qdrant_agent_tools)]
)

# Define system prompt used during agent creation
qdrant_agent_prompt_template = PromptTemplate.from_template(
    """
Role:
You are the Qdrant Search Tool, a micro-service that combines high-fidelity vector embeddings with rich, structured metadata filtering to retrieve the most relevant records from a Qdrant collection.

Description & Purpose:
- Given a natural-language query, an optional dictionary of filters (exact matches or numeric ranges), and a desired result count k, your job is to:
    - Embed the query via OpenAI.
    - Translate filters into Qdrant payload conditions.
    - Execute a hybrid semantic + metadata search.
    - Return the top-k hits, each with its id, similarity score, and full payload metadata.

Inputs (Parameters):
- query (string): free-text search string.
- filters (dict): e.g.
    Exact: {{ "state": "delhi", "industry_sector": "saas" }}
    Range: {{ "year_founded": {{"gte": 2000, "lte": 2010}} }}
- k (integer): the number of top results to return.

Examples:
Pure semantic (no filters)
qdrant_search(
    query="emerging agritech startups",
    filters=None,
    k=5
)
# → returns top-5 agritech vectors by relevance

Metadata only
qdrant_search(
    query="",
    filters={{ "state": "karnataka", "industry_sector": "fintech" }},
    k=10
)
# → returns any fintech startups in Karnataka, ordered by vector‐default rank

Hybrid (semantic + filters + range)
qdrant_search(
    query="best B2B platforms",
    filters={{ 
        "state": "delhi",
        "year_founded": {{"gte": 2015}},
        "industry_sector": "saas"
    }},
    k=3
)
# → returns top-3 SaaS B2B startups in Delhi founded ≥2015

Guidelines & Constraints:
- Must apply both vector similarity and all payload filters.
- For textual filters use exact keyword match.
- For numeric filters support gte / lte semantics.
- If filters=None, perform a pure semantic lookup.
- Always return at most k results.
- Never omit an entry’s payload.
- Ensure consistent lower-casing of filter values and field names.
- Never return more than k results, even if multiple entries have the same score.
- If no results match, return an empty list [].

**Output Format:**
- The final answer MUST be a valid Python dictionary (not a JSON string).
- Do NOT wrap the output in quotes or format as a string.
- Example:
    Final Answer: [
        {{
            "id": "company_123",
            "score": 0.92,
            "payload": {{
                "company_name": "Acme Corp",
                "industry_sector": "saas",
                ...
            }}
        }},
        ...
    ]

Format:
Question: the input query
Thought: think step-by-step about what to extract
Action: the tool to use, from [{tool_names}]
Action Input: JSON string or plain text input to the tool
Observation: result returned by the tool
... (repeat Thought/Action/Observation as needed)
Thought: I have gathered all necessary structured data.
Final Answer: a Python dictionary of all extracted metadata and filters

Constraints:
- NEVER ask the user again
- ONLY use tools
- NEVER hallucinate missing data

Begin!

Question: {input}
{agent_scratchpad}
"""
)


# Format the prompt with tool descriptions and names
formatted_prompt = qdrant_agent_prompt_template.partial(
    tools=tool_help_text,
    tool_names=", ".join(tool.name for tool in qdrant_agent_tools),
)

# 🔧 Define the React-style agent
llm = ChatOpenAI(model="gpt-4o") 


# Create the agent
qdrant_agent = create_react_agent(
    llm=llm,
    tools=qdrant_agent_tools,
    prompt=formatted_prompt,
)


In [ ]:
from langchain.agents import AgentExecutor

executor = AgentExecutor(agent=qdrant_agent, 
                         tools=qdrant_agent_tools, 
                         verbose=True, 
                         handle_parsing_errors=True)

import json
structured_input = {
    "query": "D2C companies with Sequoia or Accel as lead investors and more than 200 employees",
    "filters": {
        "product_categories": "d2c",
        
        },
    "k": 5
}


# 3️⃣ Invoke!
result = executor.invoke({"input": structured_input})
print(result["output"])



In [10]:
# src/nodes/quadrant_search_node.py

from typing import Literal, List, Any
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import HumanMessage
from langgraph.types import Command
from schema.agent_state import AgentState

# Make sure you have already created your React‑style Qdrant agent somewhere:
# from src.agents.qdrant_search_agent import qdrant_search_agent

def quadrant_search_node(state: AgentState) -> Command[Literal["__end__"]]:
    """
    React‑style Qdrant Search Node:
    
    1. Reads `enhanced_query`, `filters`, and `k` from AgentState.
    2. Streams the `qdrant_search_agent` (verbose=False) to capture:
       - AgentAction steps (which tool it called)
       - Intermediate observations (tool outputs)
       - Final AgentFinish output (a Python dict)
    3. Parses that dict into:
       - `results`        → list of hit dictionaries
       - `reasoning`      → agent’s summary of what it did
    4. Appends a full trace into `actions` and `observations`
    5. Updates AgentState with:
       • `messages` (appends the raw dict as a HumanMessage)  
       • `retrieved_results` (the list of hits)  
       • `final_response` (the reasoning string)  
       • `actions` & `observations` (full logs)  
       • `agent_name` set to `"qdrant_search"`  
    6. Returns `goto="__end__"` to terminate the workflow.
    """

    # ─── Disable built‑in verbose logging so we can capture our own logs ────
    qdrant_search_agent.verbose = False

    # ─── Prepare log collectors for this node invocation ────────────────────
    actions: List[str] = []
    observations: List[str] = []
    final_output: Any = None

    # ─── Build the payload for the React‑style agent ────────────────────────
    payload = {
        "messages": state.get("messages", []),
        "enhanced_query": state.get("enhanced_query"),
        "filters": state.get("filters"),
        "k": state.get("k"),
    }

    # ─── Stream through the agent’s Thought→Action→Observation loop ────────
    for step in qdrant_search_agent.stream(payload):
        if isinstance(step, AgentAction):
            # Agent decided on a tool call or its own internal reasoning step
            actions.append(str(step.log))
        elif isinstance(step, AgentFinish):
            # Agent has finished; expect a Python dict in `output`
            final_output = step.return_values.get("output")
        else:
            # Intermediate tool output (e.g. the raw DB results or metadata)
            observations.append(str(step))

    # ─── Parse the final output dict (or fall back to empty) ───────────────
    if isinstance(final_output, dict):
        results = final_output.get("results", [])
        reasoning = final_output.get("reasoning", "")
        message_content = str(final_output)
    else:
        # Fallback if no dict was returned
        results = []
        reasoning = ""
        message_content = str(final_output) if final_output is not None else ""

    # ─── Append a completion marker and summary to the logs ────────────────
    new_actions = (
        state.get("actions", [])
        + actions
        + ["Qdrant Search agent completed"]
    )
    new_observations = (
        state.get("observations", [])
        + observations
        + [
            f"Results count: {len(results)}",
            f"Reasoning: {reasoning}",
        ]
    )

    # ─── Return the updated AgentState and signal graph termination ────────
    return Command(
        update={
            # 1) Append the raw final_output dict as a chat message
            "messages": [
                HumanMessage(content=message_content, name="qdrant_search")
            ],
            # 2) Store structured results and reasoning
            "retrieved_results": results,
            "final_response": reasoning,
            # 3) Persist full trace logs
            "actions": new_actions,
            "observations": new_observations,
            # 4) Label which agent just ran
            "agent_name": "qdrant_search",
        },
        goto="__end__"  # Terminates the StateGraph
    )
